In [4]:
import numpy as np
from pprint import pprint

import os 
from pathlib import Path

dir_path = str(Path(os.getcwd()).parent.absolute())
print(dir_path)

import sys
sys.path.append(dir_path)
if 'dwsimopt.sim_opt' in sys.modules:  # Is the module in the register?
    del sys.modules['dwsimopt.sim_opt']  # If so, remove it.
    del SimulationOptimization
from dwsimopt.sim_opt import SimulationOptimization

c:\Users\lfsfr\Desktop\pyDWSIMopt


In [5]:
# Getting DWSIM path from system path
for k,v in enumerate(os.environ['path'].split(';')):
    if v.find('\DWSIM')>-1:
        path2dwsim = os.path.join(v, '')
if path2dwsim == None:
    path2dwsim = "C:\\Users\\lfsfr\\AppData\\Local\\DWSIM7\\"   #insert manuall
print(path2dwsim)

# Loading DWSIM simulation into Python (Simulation object)
sim_smr = SimulationOptimization(dof=np.array([]), path2sim= os.path.join(dir_path, "examples\\SMR_LNG\\SMR.dwxmz"), 
                     path2dwsim = path2dwsim)
sim_smr.savepath = os.path.join(dir_path, "examples\\SMR_LNG\\SMR2.dwxmz")
sim_smr.add_refs()

# Instanciate automation manager object
from DWSIM.Automation import Automation2
if ('interf' not in locals()):    # create automation manager
    interf = Automation2()

# Connect simulation in sim.path2sim
sim_smr.connect(interf)

C:\Users\lfsfr\AppData\Local\DWSIM7\
added refs
Simulation was loaded successfully


In [6]:
from dwsimopt.py2dwsim import * 

# Assign DoF:
create_pddx( ["MR-1", "CompoundMassFlow", "Nitrogen", "kg/s"],    sim_smr, element="dof" )
create_pddx( ["MR-1", "CompoundMassFlow", "Methane", "kg/s"],     sim_smr, element="dof" )
create_pddx( ["MR-1", "CompoundMassFlow", "Ethane", "kg/s"],      sim_smr, element="dof" )
create_pddx( ["MR-1", "CompoundMassFlow", "Propane", "kg/s"],     sim_smr, element="dof" )
create_pddx( ["MR-1", "CompoundMassFlow", "Isopentane", "Pa"],    sim_smr, element="dof" )
create_pddx( ["VALV-01", "OutletPressure", "Mixture", "Pa"],      sim_smr, element="dof" )
create_pddx( ["COMP-1", "OutletPressure", "Mixture", "Pa"],       sim_smr, element="dof" )
create_pddx( ["COOL-08", "OutletTemperature", "Mixture", "K"],    sim_smr, element="dof" )

# Assign F
create_pddx( ["Sum_W", "EnergyFlow", "Mixture", "kW"], sim_smr, element="fobj" )

# adding constraints (g_i <= 0):
g1 = create_pddx( ["MITA1-Calc", "OutputVariable", "mita", "°C"], sim_smr, element="constraint", assign=False )
assign_pddx( lambda: 3-g1[0]() , ["MITA1-Calc", "OutputVariable", "mita", "°C"], sim_smr, element="constraint" )
g2 = create_pddx( ["MITA2-Calc", "OutputVariable", "mita", "°C"], sim_smr, element="constraint", assign=False )
assign_pddx( lambda: 3-g2[0]() , ["MITA2-Calc", "OutputVariable", "mita", "°C"], sim_smr, element="constraint" )


pprint(vars(sim_smr))

{'dof': array([[<function toDwsim.<locals>.<lambda> at 0x000001B373A0F9D0>,
        'MR-1', 'CompoundMassFlow', 'Nitrogen', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FC10>,
        'MR-1', 'CompoundMassFlow', 'Methane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FB80>,
        'MR-1', 'CompoundMassFlow', 'Ethane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FA60>,
        'MR-1', 'CompoundMassFlow', 'Propane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0F940>,
        'MR-1', 'CompoundMassFlow', 'Isopentane', 'Pa'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0F3A0>,
        'VALV-01', 'OutletPressure', 'Mixture', 'Pa'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FAF0>,
        'COMP-1', 'OutletPressure', 'Mixture', 'Pa'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0F160>,
        'COOL-08', 'OutletTemperature', 'Mixture', 'K']], 

In [7]:
# Initial simulation optimization setup
# Initial guess of optimization
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -60+273.15] )

# Testing for simulation at x0
sim_smr.calculate_optProblem(1.0*x0)
print(sim_smr.x_val,
      sim_smr.f_val,
      sim_smr.g_val)

# Test saving simulation at x0 in 'savepath'
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.savepath,True)

# Inspecting simulation object
pprint(vars(sim_smr))

f = [0.38910439], g = [-1.78887881 -3.47846719] at x = [6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.13150000e+02]
[6.94444444e-05 1.94444444e-04 2.77777778e-04 3.05555556e-04
 5.00000000e-04 2.50000000e+05 5.00000000e+06 2.13150000e+02] [0.38910439] [-1.78887881 -3.47846719]
{'dof': array([[<function toDwsim.<locals>.<lambda> at 0x000001B373A0F9D0>,
        'MR-1', 'CompoundMassFlow', 'Nitrogen', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FC10>,
        'MR-1', 'CompoundMassFlow', 'Methane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FB80>,
        'MR-1', 'CompoundMassFlow', 'Ethane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0FA60>,
        'MR-1', 'CompoundMassFlow', 'Propane', 'kg/s'],
       [<function toDwsim.<locals>.<lambda> at 0x000001B373A0F940>,
        'MR-1', 'CompoundMassFlow', 'Isopentane', 'Pa'],
       [<function toDwsim.<loca

In [5]:
# Setup for optimization
# convergence tolerances
xtol=0.01
ftol=0.01
maxiter=5 # +- 20 seconds per iteration

# decision variables bounds
bounds_raw = np.array( [0.5*np.asarray(x0), 1.5*np.asarray(x0)] )   # 50 % around base case
bounds_raw[0][-1] = 153     # precool temperature low limit manually
bounds_raw[1][-1] = 253     # precool temperature upper limit manually

# regularizer calculation
regularizer = np.zeros(x0.size)
import math
for i in range(len(regularizer)):
    regularizer[i] = 10**(-1*math.floor(math.log(x0[i],10))) # regularizer for magnitude order of 1e0

# bounds regularized
bounds_reg = regularizer*bounds_raw
# bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])

# objective and constraints lambda definitions
f = lambda x: sim_smr.calculate_optProblem(np.asarray(x)/regularizer)[0:sim_smr.n_f]
g = lambda x: sim_smr.calculate_optProblem(np.asarray(x)/regularizer)[sim_smr.n_f:(sim_smr.n_f+sim_smr.n_g)]
# nonlinear_constraint = optimize.NonlinearConstraint(g, -np.inf, 0, jac='2-point', hess=optimize.BFGS())

In [6]:
# Global optimization with PSO
from sko.PSO import PSO

# f_pen = lambda x: fpen_barrier(sim_smr,x/regularizer)
result_pso = PSO(func= lambda x: sim_smr.fpen_barrier(x/regularizer), n_dim=sim_smr.n_dof, pop=2*sim_smr.n_dof, max_iter=50, lb=bounds_reg[0], ub=bounds_reg[1], verbose=True)
result_pso.record_mode = True
if sim_smr.n_f > 1:
    print("Multi-objective optimization not supported (yet)")
elif sim_smr.n_f < 1:
    print("Invalid number of objective functions")
else:
    print("Starting global optimization")
    result_pso.run()

f = [0.30220674], g = [17.31570443  1.28512746] at x = [6.02599338e-05 1.11493372e-04 2.74144662e-04 2.53265652e-04
 3.46520273e-04 3.32185739e+05 6.16840449e+06 1.63246581e+02]
f = [0.42809863], g = [26.05349287 -1.31824068] at x = [5.99241625e-05 2.24288466e-04 3.25794382e-04 3.34464106e-04
 2.54303905e-04 3.58008640e+05 6.74108219e+06 2.51823289e+02]
f = [0.33227062], g = [ 2.4016242  -1.26416533] at x = [9.18781869e-05 9.90976112e-05 3.65757712e-04 2.51835967e-04
 5.69371097e-04 2.42843107e+05 3.81870010e+06 2.19084483e+02]
f = [0.33968959], g = [16.19148811  4.13193897] at x = [8.83086155e-05 1.90249471e-04 1.86968188e-04 1.71497083e-04
 4.90565825e-04 2.48334735e+05 3.03372642e+06 1.55336242e+02]
f = [0.31956271], g = [1.35533723 1.35922093] at x = [4.66667122e-05 1.05691075e-04 3.38832406e-04 2.96108306e-04
 3.72915738e-04 2.67546729e+05 6.17936373e+06 1.83092447e+02]
f = [0.29383034], g = [43.46580041  3.81098297] at x = [7.95147671e-05 1.22082607e-04 2.31830199e-04 1.80473518e